# Data preprocessing

### Import Libraries

In [1]:
import numpy as np
import pandas as pd

import json

import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller

import matplotlib.pyplot as plt
import matplotlib.cm as cm

# Import Datetime and the Pandas DataReader
from datetime import datetime
from pandas_datareader import data, wb

# Import scikit instruments
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn import preprocessing
from sklearn.metrics import silhouette_score

# just set the seed for the random number generator
np.random.seed(107)

/Users/simaomoraessarmento/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
from tiingo import TiingoClient

config = {}

# To reuse the same HTTP Session across API calls (and have better performance), include a session key.
config['session'] = True

# If you don't have your API key as an environment variable,
# pass it in via a configuration dictionary.
config['api_key'] = "fd7689ea0019b4292b78d3efd6f7bb9c896083ff"

# Initialize
client = TiingoClient(config)

### Import Classes

In [6]:
%load_ext autoreload
%aimport class_SeriesAnalyser, class_Trader, class_DataProcessor
%autoreload 1

In [7]:
data_processor = class_DataProcessor.DataProcessor()

# Preprocessing Commodity ETFs Dataset

We proceed to retrieve the time series from the identified ETFs, and save the data retrieved in a Datframe.

1. Upload time series corresponding to ETFs and store them in a dictionary
2. Convert the dictionary in a Dataframe by concatenating time series
3. Save Dataframe in pickle file


In [40]:
etfs, etfs_unique, tickers = data_processor.read_ticker_excel(path='data/etfs/commodity_ETFs_long_updated.xlsx')

In [41]:
duplicated_etfs = etfs[etfs.duplicated(subset=['Ticker'], keep=False)].sort_values(['Ticker'])
duplicated_etfs

,Sector,Ticker,Description,Issuer,AUM,ExpenseRatio,3-MO TR,Segment
224,Gold,AAAU,Perth Mint Physical Gold ETF,Exchange Traded Concepts,$105.42M,0.18%,NaN,Commodities: Precious Metals Gold
335,Precious Metals,AAAU,Perth Mint Physical Gold ETF,Exchange Traded Concepts,$105.42M,0.18%,NaN,Commodities: Precious Metals Gold
56,Commodities,AAAU,Perth Mint Physical Gold ETF,Exchange Traded Concepts,$105.42M,0.18%,NaN,Commodities: Precious Metals Gold
359,Silver,AGQ,ProShares Ultra Silver,ProShares,$172.80M,5.28%,NaN,Leveraged Commodities: Precious Metals Silver
331,Precious Metals,AGQ,ProShares Ultra Silver,ProShares,$172.80M,5.28%,NaN,Leveraged Commodities: Precious Metals Silver
46,Commodities,AGQ,ProShares Ultra Silver,ProShares,$172.80M,5.28%,NaN,Leveraged Commodities: Precious Metals Silver
141,Cotton,BAL,iPath Series B Bloomberg Cotton Subindex Total...,Barclays Bank PLC,$8.60M,0.45%,3.64%,Commodities: Agriculture Cotton
102,Commodities,BAL,iPath Series B Bloomberg Cotton Subindex Total...,Barclays Bank PLC,$8.60M,0.45%,3.64%,Commodities: Agriculture Cotton
10,Agriculture,BAL,iPath Series B Bloomberg Cotton Subindex Total...,Barclays Bank PLC,$8.60M,0.45%,3.64%,Commodities: Agriculture Cotton
326,Precious Metals,BAR,GraniteShares Gold Trust,GraniteShares,$444.96M,0.17%,NaN,Commodities: Precious Metals Gold


In [42]:
etfs_unique.Segment.value_counts()

Equity: U.S. MLPs                                            25
Commodities: Broad Market                                    21
Commodities: Precious Metals Gold                            12
Equity: U.S. Energy                                          11
Commodities: Energy Crude Oil                                 9
Equity: Global Gold Miners                                    7
Inverse Commodities: Energy Crude Oil                         6
Commodities: Agriculture                                      6
Equity: Global Water                                          5
Leveraged Commodities: Energy Crude Oil                       5
Equity: Global Metals & Mining                                5
Commodities: Precious Metals Silver                           5
Commodities: Energy                                           4
Inverse Commodities: Precious Metals Gold                     4
Equity: U.S. Oil & Gas Exploration & Production               3
Leveraged Equity: U.S. MLPs             

**Let's see how this dataset compares with the one proposed when we select solely the commodity ETFs**

In [43]:
etfs_small, etfs_unique_small, tickers_small = \
                data_processor.read_ticker_excel(path='data/etfs/commodity_ETFs.xlsx')

In [44]:
not_overlappping = pd.Series()
for sample in etfs_unique.Ticker:
    if sample not in etfs_small.Ticker.values:
        not_overlappping = pd.concat([not_overlappping, etfs_unique[etfs_unique.Ticker == sample].Segment])

In [45]:
not_overlappping.value_counts()

Equity: U.S. MLPs                                            25
Equity: U.S. Energy                                          11
Equity: Global Gold Miners                                    7
Equity: Global Metals & Mining                                5
Equity: Global Water                                          5
Leveraged Equity: U.S. MLPs                                   3
Equity: U.S. Oil & Gas Exploration & Production               3
Leveraged Equity: Global Gold Miners                          3
Inverse Equity: U.S. Oil & Gas                                3
Equity: U.S. Oil & Gas Equipment & Services                   3
Inverse Equity: U.S. Energy                                   3
Inverse Equity: Global Gold Miners                            3
Equity: Global Silver Miners                                  3
Equity: Global Oil & Gas                                      2
Leveraged Equity: U.S. Energy                                 2
Equity: Global Nuclear Energy           

In [46]:
print(len(not_overlappping))
print(len(etfs_unique)-len(etfs_small))

98
98


The broader category contains ETFs that are not directly investing in commodities but in related companies instead. These look like a good investment opportunity and therefore we will guide ourselves through the broader group. We will discard those that are not possible to retrieve.

As for downloading the data, we define a wide interval, from 2000 to 2019. Those ETFs not trading during the entire interval will automatically fill with np.Nan the non-trading days.

In [50]:
tickers = list(tickers)

In [52]:
error_counter = 0
dataset_tiingo = {key: None for key in tickers}
for ticker in tickers:
    try:
        df = client.get_dataframe([ticker],
                                  frequency='daily',
                                  metric_name='adjClose',
                                  startDate='2000-01-01',
                                  endDate='2019-01-01')
        series = df[ticker]
        series.name = ticker  # filter close price only
        dataset_tiingo[ticker] = series.copy()
    except:
        error_counter = error_counter + 1
        print('Not Possible to retrieve information for ' + ticker)

print('\nUnable to download ' + str(error_counter / len(tickers) * 100) + '% of the ETFs')

Not Possible to retrieve information for NRGD
Not Possible to retrieve information for NRGO
Not Possible to retrieve information for NRGU
Not Possible to retrieve information for NRGZ
Not Possible to retrieve information for YGRN

Unable to download 2.3474178403755865% of the ETFs


**2) After having downloaded all time series, one by one, we merge in a dataframe:**

In [57]:
df_prices, _ = data_processor.dict_to_df(dataset_tiingo)

**3) Save Dataframe in pickle file:**

In [59]:
df_prices.to_pickle('data/etfs/pickle/commodity_ETFs_long_updated')